In [ ]:
# =========================
# Colab: PDF crawler + downloader + manifest
# Fix: Waits for download event after "Download is starting" exception to avoid race conditions
# =========================

!pip -q install beautifulsoup4 tqdm requests playwright nest_asyncio
!playwright install --with-deps chromium

import os, re, json, time, hashlib, asyncio
import nest_asyncio
from dataclasses import dataclass, asdict
from collections import deque, Counter
from urllib.parse import urlparse, urljoin, urldefrag, urlunparse, parse_qsl, urlencode

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError

nest_asyncio.apply()

# =========================
# CONFIG
# =========================
START_URL = "https://www.technology1.com/company/investors/annual-reports"
MAX_DEPTH = 0
MAX_PDFS  = 200
MAX_PAGES = 200

OUTPUT_DIR    = "/content/pdfs"
MANIFEST_PATH = "/content/manifest.json"

CRAWL_SAME_DOMAIN_ONLY = True
HTML_ALLOWED_PREFIX    = "/company/investors"
ALLOW_OFFDOMAIN_PDFS   = True

REQUEST_TIMEOUT   = 30
REQUEST_DELAY_SEC = 0.10
VERIFY_TLS        = True

DEBUG = True
DEBUG_MAX_PAGES_PRINT = 2
DEBUG_PDF_FIRST_N = 3  # print debug for first N PDF downloads

BROWSER_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
}

PDF_MAGIC = b"%PDF"
TRACKING_PARAMS = {"utm_source","utm_medium","utm_campaign","utm_term","utm_content","gclid","fbclid"}

# =========================
# Helpers
# =========================
def ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)

def utc_now_iso() -> str:
    return time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())

def sha256_bytes(b: bytes) -> str:
    return hashlib.sha256(b).hexdigest()

def safe_filename(name: str, max_len: int = 180) -> str:
    name = re.sub(r"[^\w\-. ]+", "_", name).strip()
    if len(name) > max_len:
        root, ext = os.path.splitext(name)
        name = root[: max_len - len(ext) - 1] + "_" + ext
    return name or "document.pdf"

def normalize_url(url: str, base: str) -> str | None:
    if not url:
        return None
    url = url.strip()
    if url.startswith(("mailto:", "tel:", "javascript:", "data:")):
        return None
    abs_url = urljoin(base, url)
    abs_url, _ = urldefrag(abs_url)
    p = urlparse(abs_url)
    if p.scheme not in ("http", "https"):
        return None
    q = [(k, v) for (k, v) in parse_qsl(p.query, keep_blank_values=True) if k not in TRACKING_PARAMS]
    q.sort(key=lambda kv: (kv[0], kv[1]))
    new_query = urlencode(q, doseq=True)
    p2 = p._replace(scheme=p.scheme.lower(), netloc=p.netloc.lower(), query=new_query)
    return urlunparse(p2)

def looks_like_pdf(url: str) -> bool:
    path = urlparse(url).path.lower()
    return path.endswith(".pdf") or "/__data/assets/pdf_file/" in path or ".pdf" in path

def html_in_scope(url: str, start_parsed) -> bool:
    p = urlparse(url)
    if CRAWL_SAME_DOMAIN_ONLY and p.netloc.lower() != start_parsed.netloc.lower():
        return False
    if HTML_ALLOWED_PREFIX and not p.path.startswith(HTML_ALLOWED_PREFIX):
        return False
    return True

def pdf_in_scope(url: str, start_parsed) -> bool:
    if ALLOW_OFFDOMAIN_PDFS:
        return True
    return urlparse(url).netloc.lower() == start_parsed.netloc.lower()

def pick_filename_from_url(url: str) -> str:
    path = urlparse(url).path
    base = os.path.basename(path) or "document.pdf"
    if not base.lower().endswith(".pdf"):
        base += ".pdf"
    return safe_filename(base)

def extract_links(soup: BeautifulSoup):
    links = set()
    for tag in soup.find_all("a", href=True):
        links.add(tag.get("href"))
    for tag in soup.find_all(["iframe", "embed"], src=True):
        links.add(tag.get("src"))
    for tag in soup.find_all("object", data=True):
        links.add(tag.get("data"))
    for tag in soup.find_all("link", href=True):
        links.add(tag.get("href"))
    return [x for x in links if x]

def is_cloudflare_interstitial(html: str) -> bool:
    if not html:
        return False
    s = html.lower()
    return ("just a moment" in s) and ("cloudflare" in s or "/cdn-cgi/" in s)

def run_coro(coro):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(coro)

def make_requests_session() -> requests.Session:
    s = requests.Session()
    s.headers.update(BROWSER_HEADERS)
    return s

# =========================
# Manifest record
# =========================
@dataclass
class PdfRecord:
    # required
    source_url: str
    local_path: str | None = None
    downloaded_at: str | None = None
    http_status: int | None = None
    # optional but recommended
    sha256: str | None = None
    content_length: int | None = None
    # diagnostics
    discovered_on: str | None = None
    final_url: str | None = None
    content_type: str | None = None
    status: str = "pending"   # downloaded|skipped|failed|pending
    reason: str | None = None
    duplicate_of: str | None = None

# =========================
# HTML Fetch (requests -> Playwright fallback)
# =========================
def fetch_html_requests(url: str, sess: requests.Session):
    try:
        resp = sess.get(url, timeout=REQUEST_TIMEOUT, allow_redirects=True, verify=VERIFY_TLS)
        ct = (resp.headers.get("Content-Type") or "").lower()
        return True, resp.status_code, resp.url, ct, resp.text, "requests", None
    except Exception as e:
        return False, None, url, "", "", "requests_error", f"{type(e).__name__}: {str(e)[:220]}"

async def fetch_html_playwright(url: str):
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(
                headless=True,
                args=["--disable-dev-shm-usage", "--no-sandbox", "--disable-setuid-sandbox"]
            )
            context = await browser.new_context(
                user_agent=BROWSER_HEADERS["User-Agent"],
                locale="en-US",
                viewport={"width": 1280, "height": 800},
                ignore_https_errors=True
            )
            page = await context.new_page()
            resp = await page.goto(url, wait_until="domcontentloaded", timeout=60000)
            await page.wait_for_timeout(2000)
            final_url = page.url
            status = resp.status if resp is not None else 200
            html = await page.content()
            await context.close()
            await browser.close()
            return True, status, final_url, "text/html", html, "playwright", None
    except PlaywrightTimeoutError as e:
        return False, 408, url, "text/html", "", "playwright_timeout", f"timeout: {str(e)[:220]}"
    except Exception as e:
        return False, None, url, "text/html", "", "playwright_error", f"{type(e).__name__}: {str(e)[:220]}"

def fetch_html(url: str, sess: requests.Session):
    ok, status, final_url, ct, html, method, err = fetch_html_requests(url, sess)
    if (status in (403, 429)) or is_cloudflare_interstitial(html):
        return run_coro(fetch_html_playwright(url))
    return ok, status, final_url, ct, html, method, err

# =========================
# Crawl
# =========================
def crawl_and_collect_pdfs(start_url: str):
    start_parsed = urlparse(start_url)
    visited_pages = set()
    seen_links = set()
    pdf_candidates: dict[str, PdfRecord] = {}

    q = deque([(start_url, 0)])
    sess = make_requests_session()
    crawl_failures = []
    debug_printed = 0

    while q and len(visited_pages) < MAX_PAGES:
        page_url, depth = q.popleft()
        if depth > MAX_DEPTH or page_url in visited_pages:
            continue

        visited_pages.add(page_url)
        time.sleep(REQUEST_DELAY_SEC)

        ok, status, final_url, ct, html, method, err = fetch_html(page_url, sess)

        if DEBUG and debug_printed < DEBUG_MAX_PAGES_PRINT:
            debug_printed += 1
            print("\n=== CRAWL DEBUG ===")
            print("Requested:", page_url)
            print("Final URL:", final_url)
            print("Depth:", depth)
            print("OK:", ok)
            print("HTTP:", status, f"({method})")
            print("Content-Type:", ct or "text/html")
            print("HTML length:", len(html or ""))
            print("Cloudflare interstitial?:", is_cloudflare_interstitial(html))
            if err:
                print("Error message:", err)
            snippet = (html or "")[:220].replace("\n", " ")
            print("Body snippet:", snippet)
            print("===================")

        if not ok or (status is not None and status >= 400):
            reason = "blocked_by_cloudflare" if is_cloudflare_interstitial(html) else "crawl_http_error"
            if method.startswith("playwright") and err:
                reason = "playwright_failed"
            crawl_failures.append({"url": page_url, "http_status": status, "reason": reason, "details": err})
            continue

        soup = BeautifulSoup(html, "html.parser")
        raw_links = extract_links(soup)

        for href in raw_links:
            norm = normalize_url(href, base=final_url)
            if not norm or norm in seen_links:
                continue
            seen_links.add(norm)

            if looks_like_pdf(norm):
                if pdf_in_scope(norm, start_parsed):
                    pdf_candidates.setdefault(norm, PdfRecord(source_url=norm, discovered_on=final_url))
                continue

            if depth < MAX_DEPTH and html_in_scope(norm, start_parsed):
                q.append((norm, depth + 1))

    return pdf_candidates, len(visited_pages), crawl_failures

# =========================
# Download PDFs via Playwright navigation (UPDATED FIX)
# =========================
async def download_pdfs_via_navigation(urls: list[str], pdf_candidates: dict[str, PdfRecord], output_dir: str):
    ensure_dir(output_dir)

    downloaded_by_final_url = set()
    downloaded_by_hash: dict[str, str] = {}
    results: list[PdfRecord] = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=["--disable-dev-shm-usage", "--no-sandbox", "--disable-setuid-sandbox"]
        )
        context = await browser.new_context(
            user_agent=BROWSER_HEADERS["User-Agent"],
            locale="en-US",
            ignore_https_errors=True,
            accept_downloads=True
        )
        page = await context.new_page()
        page.set_default_navigation_timeout(60000)

        # Land on START_URL first (get cookies/session).
        try:
            await page.goto(START_URL, wait_until="domcontentloaded")
            await page.wait_for_timeout(1200)
        except Exception as e:
            print(f"Initial setup navigation warning: {e}")

        for idx, url in enumerate(tqdm(urls, desc="Downloading PDFs (browser navigation)"), start=1):
            rec = pdf_candidates[url]
            rec.downloaded_at = utc_now_iso()

            try:
                # 1. Prepare to catch a download event
                download_future = asyncio.Future()

                def on_download(download):
                    if not download_future.done():
                        download_future.set_result(download)

                page.on("download", on_download)

                # 2. Referer trick
                try:
                    await page.goto(START_URL, wait_until="domcontentloaded")
                except:
                    pass

                # 3. Go to the actual PDF URL
                resp = None
                browser_download = None

                try:
                    resp = await page.goto(url, wait_until="domcontentloaded")
                    await page.wait_for_timeout(300)
                except Exception as e:
                    if "Download is starting" in str(e):
                        # Fix: If download is starting, we MUST wait for the future
                        # because it might not be 'done' yet in the microtask queue.
                        try:
                            browser_download = await asyncio.wait_for(download_future, timeout=30.0)
                        except asyncio.TimeoutError:
                            rec.reason = "Download event timed out (captured 'Download is starting' but no event)"
                    else:
                        rec.reason = f"Nav Error: {str(e)[:100]}"

                page.remove_listener("download", on_download)

                # 4. Handle results
                content_body = b""

                # Check if we got a download via the future (either from exception block or concurrent success)
                if not browser_download and download_future.done():
                    browser_download = download_future.result()

                if browser_download:
                    try:
                        # Wait for download to finish on disk
                        dl_path = await browser_download.path()
                        if dl_path:
                            with open(dl_path, "rb") as f:
                                content_body = f.read()
                        rec.http_status = 200
                        rec.final_url = browser_download.url
                        rec.content_type = "application/pdf"
                    except Exception as e:
                        rec.reason = f"Download file read error: {e}"
                        rec.status = "failed"
                        results.append(rec)
                        continue

                # Otherwise, check if we have a standard page response (Inline PDF)
                elif resp:
                    rec.http_status = resp.status
                    rec.final_url = page.url
                    rec.content_type = (resp.headers.get("content-type") or "").lower()

                    if rec.http_status < 400:
                        content_body = await resp.body()

                # 5. Process the Content (Common Logic)
                if not content_body:
                    if not rec.reason:
                        rec.reason = f"No content retrieved (HTTP {rec.http_status})"
                    rec.status = "failed"
                    results.append(rec)
                    continue

                # Magic bytes check
                is_pdf_by_magic = content_body.lstrip().startswith(PDF_MAGIC)
                is_pdf_by_header = "application/pdf" in (rec.content_type or "")

                if not (is_pdf_by_magic or is_pdf_by_header):
                    rec.status = "skipped"
                    rec.reason = "Not a PDF (content-type + magic-bytes failed)"
                    results.append(rec)
                    continue

                # Dedupe by Final URL
                if rec.final_url in downloaded_by_final_url:
                    rec.status = "skipped"
                    rec.reason = "Duplicate final URL"
                    results.append(rec)
                    continue

                rec.content_length = len(content_body)
                rec.sha256 = sha256_bytes(content_body)

                # Dedupe by Hash
                if rec.sha256 in downloaded_by_hash:
                    rec.status = "skipped"
                    rec.reason = "Duplicate content (sha256 match)"
                    rec.duplicate_of = downloaded_by_hash[rec.sha256]
                    results.append(rec)
                    continue

                # Save file
                fname = pick_filename_from_url(rec.final_url or url)
                out_path = os.path.join(output_dir, fname)
                if os.path.exists(out_path):
                    root, ext = os.path.splitext(out_path)
                    out_path = f"{root}_{hashlib.md5((rec.final_url or url).encode()).hexdigest()[:8]}{ext}"

                with open(out_path, "wb") as f:
                    f.write(content_body)

                rec.local_path = out_path
                rec.status = "downloaded"

                downloaded_by_hash[rec.sha256] = out_path
                downloaded_by_final_url.add(rec.final_url)

                if DEBUG and idx <= DEBUG_PDF_FIRST_N:
                    print("\n=== PDF DOWNLOAD DEBUG ===")
                    print("Source:", rec.source_url)
                    print("Mode  :", "Attachment" if browser_download else "Inline")
                    print("Bytes :", rec.content_length)
                    print("SHA256:", rec.sha256[:16], "...")
                    print("==========================")

                results.append(rec)

            except Exception as e:
                rec.status = "failed"
                rec.reason = f"Exception: {type(e).__name__}: {str(e)[:120]}"
                results.append(rec)

        await context.close()
        await browser.close()

    return results

# =========================
# Manifest + summary
# =========================
def write_manifest(path: str, start_url: str, pages_visited: int, pdf_candidates: dict, records: list[PdfRecord], crawl_failures):
    manifest = {
        "run": {
            "start_url": start_url,
            "max_depth": MAX_DEPTH,
            "max_pdfs": MAX_PDFS,
            "pages_visited": pages_visited,
            "pdfs_found": len(pdf_candidates),
            "pdfs_downloaded": sum(1 for r in records if r.status == "downloaded"),
            "download_failures": sum(1 for r in records if r.status == "failed"),
            "crawl_failures": len(crawl_failures),
            "timestamp_utc": utc_now_iso(),
        },
        "items": [asdict(r) for r in records],
        "crawl_failures": crawl_failures,
    }
    with open(path, "w", encoding="utf-8") as f:
        json.dump(manifest, f, indent=2, ensure_ascii=False)
    return manifest

# =========================
# RUN
# =========================
ensure_dir(OUTPUT_DIR)

pdf_candidates, pages_visited, crawl_failures = crawl_and_collect_pdfs(START_URL)

print("\n=== PDF CANDIDATES DEBUG ===")
print("Total candidates:", len(pdf_candidates))
for i, (u, rec) in enumerate(list(pdf_candidates.items())[:12], start=1):
    print(f"{i}. {u} (discovered_on={rec.discovered_on})")

urls = list(pdf_candidates.keys())[:MAX_PDFS]
records = run_coro(download_pdfs_via_navigation(urls, pdf_candidates, OUTPUT_DIR))
manifest = write_manifest(MANIFEST_PATH, START_URL, pages_visited, pdf_candidates, records, crawl_failures)

# End-of-run summary
download_failures = [r for r in records if r.status == "failed"]
crawl_fail_counts = Counter((x.get("reason", "unknown") for x in crawl_failures))
dl_fail_counts    = Counter(((r.reason or "unknown") for r in download_failures))

print("\n=== End-of-Run Summary ===")
print(f"Pages visited:     {pages_visited}")
print(f"PDFs found:        {len(pdf_candidates)}")
print(f"PDFs downloaded:   {manifest['run']['pdfs_downloaded']}")
print(f"Failures:          {manifest['run']['crawl_failures'] + manifest['run']['download_failures']}")

if crawl_failures:
    print("Crawl failures (top reasons):")
    for reason, cnt in crawl_fail_counts.most_common(5):
        print(f"  - {cnt}x {reason}")

if download_failures:
    print("Download failures (top reasons):")
    for reason, cnt in dl_fail_counts.most_common(5):
        print(f"  - {cnt}x {reason}")

print(f"\nOutput folder: {OUTPUT_DIR}")
print(f"Manifest:      {MANIFEST_PATH}")

In [ ]:
# Install required packages
!pip install pypdf2 rank-bm25 transformers torch -q

import os
import json
import pickle
import re
from typing import List, Dict
from pathlib import Path
from PyPDF2 import PdfReader
from rank_bm25 import BM25Okapi
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

# Configuration
PDF_FOLDER = "/content/pdfs"
INDEX_FOLDER = "/content/data/index"
CHUNK_SIZE = 600
CHUNK_OVERLAP = 150

class Document:
    def __init__(self, text: str, pdf: str, page: int):
        self.text = text
        self.pdf = pdf
        self.page = page

class AIRAGChatbot:
    def __init__(self):
        self.documents: List[Document] = []
        self.bm25 = None
        self.tokenized_corpus = []
        self.qa_model = None

    def load_qa_model(self):
        """Load question-answering model"""
        print("🤖 Loading AI model (this may take a minute)...")
        # Using a small, fast QA model that works well in Colab
        self.qa_model = pipeline(
            "question-answering",
            model="deepset/roberta-base-squad2",
            device=-1  # CPU
        )
        print("✓ AI model loaded")

    def extract_text_from_pdfs(self, pdf_folder: str):
        """Extract text from all PDFs with page association"""
        print(f"📚 Reading PDFs from {pdf_folder}...")

        if not os.path.exists(pdf_folder):
            raise ValueError(f"PDF folder not found: {pdf_folder}")

        pdf_files = list(Path(pdf_folder).glob("*.pdf"))
        if not pdf_files:
            raise ValueError(f"No PDF files found in {pdf_folder}")

        for pdf_path in pdf_files:
            try:
                reader = PdfReader(str(pdf_path))
                pdf_name = pdf_path.name

                for page_num, page in enumerate(reader.pages, start=1):
                    text = page.extract_text()
                    if text.strip():
                        chunks = self.chunk_text(text, CHUNK_SIZE, CHUNK_OVERLAP)
                        for chunk in chunks:
                            self.documents.append(Document(
                                text=chunk,
                                pdf=pdf_name,
                                page=page_num
                            ))

                print(f"✓ Processed: {pdf_name} ({len(reader.pages)} pages)")
            except Exception as e:
                print(f"✗ Error processing {pdf_path.name}: {e}")

        print(f"✓ Total chunks extracted: {len(self.documents)}")

    def chunk_text(self, text: str, chunk_size: int, overlap: int) -> List[str]:
        """Split text into overlapping chunks"""
        words = text.split()
        chunks = []

        for i in range(0, len(words), chunk_size - overlap):
            chunk = ' '.join(words[i:i + chunk_size])
            if chunk.strip():
                chunks.append(chunk)

        return chunks if chunks else [text]

    def build_index(self):
        """Build BM25 index"""
        print("🔨 Building BM25 index...")

        if not self.documents:
            raise ValueError("No documents to index!")

        self.tokenized_corpus = [
            doc.text.lower().split() for doc in self.documents
        ]
        self.bm25 = BM25Okapi(self.tokenized_corpus)

        print(f"✓ Index built with {len(self.documents)} chunks")

    def save_index(self, index_folder: str):
        """Persist index to disk"""
        print(f"💾 Saving index to {index_folder}...")

        os.makedirs(index_folder, exist_ok=True)

        docs_data = [
            {"text": doc.text, "pdf": doc.pdf, "page": doc.page}
            for doc in self.documents
        ]

        with open(f"{index_folder}/documents.json", "w") as f:
            json.dump(docs_data, f)

        with open(f"{index_folder}/bm25.pkl", "wb") as f:
            pickle.dump(self.bm25, f)

        print("✓ Index saved successfully")

    def load_index(self, index_folder: str):
        """Load index from disk"""
        print(f"📂 Loading index from {index_folder}...")

        with open(f"{index_folder}/documents.json", "r") as f:
            docs_data = json.load(f)

        self.documents = [
            Document(d["text"], d["pdf"], d["page"]) for d in docs_data
        ]

        with open(f"{index_folder}/bm25.pkl", "rb") as f:
            self.bm25 = pickle.load(f)

        self.tokenized_corpus = [
            doc.text.lower().split() for doc in self.documents
        ]

        print(f"✓ Index loaded with {len(self.documents)} chunks")

    def retrieve(self, query: str, top_k: int = 10) -> List[Dict]:
        """Retrieve top-k relevant documents"""
        if not self.bm25:
            raise ValueError("Index not built or loaded!")

        tokenized_query = query.lower().split()
        scores = self.bm25.get_scores(tokenized_query)
        top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]

        results = []
        for idx in top_indices:
            doc = self.documents[idx]
            results.append({
                "pdf": doc.pdf,
                "page": doc.page,
                "snippet": doc.text[:200] + "..." if len(doc.text) > 200 else doc.text,
                "full_text": doc.text,
                "score": scores[idx]
            })

        return results

    def answer_question(self, question: str, top_k: int = 10) -> Dict:
        """Answer question using AI model"""

        if not self.qa_model:
            self.load_qa_model()

        # Retrieve relevant chunks
        sources = self.retrieve(question, top_k)

        if not sources:
            return {
                "answer": "I couldn't find relevant information to answer this question.",
                "sources": []
            }

        # Combine top sources as context
        context = "\n\n".join([src["full_text"] for src in sources[:5]])

        # Truncate context if too long (model limit is ~512 tokens)
        max_context_length = 2000
        if len(context) > max_context_length:
            context = context[:max_context_length]

        try:
            # Use AI model to extract answer
            result = self.qa_model(
                question=question,
                context=context,
                max_answer_len=200,
                handle_impossible_answer=True
            )

            answer = result['answer']
            confidence = result['score']

            # If confidence is too low, return top chunk instead
            if confidence < 0.1:
                answer = sources[0]["full_text"][:400] + "..."

        except Exception as e:
            print(f"AI model error: {e}")
            answer = sources[0]["full_text"][:400] + "..."

        return {
            "answer": answer,
            "sources": [
                {
                    "pdf": src["pdf"],
                    "page": src["page"],
                    "snippet": src["snippet"]
                }
                for src in sources[:3]
            ]
        }

# Initialize chatbot
print("🚀 Initializing AI-Powered RAG Chatbot...")
chatbot = AIRAGChatbot()

# Ingest PDFs and build index
try:
    if os.path.exists(f"{INDEX_FOLDER}/bm25.pkl"):
        chatbot.load_index(INDEX_FOLDER)
    else:
        chatbot.extract_text_from_pdfs(PDF_FOLDER)
        chatbot.build_index()
        chatbot.save_index(INDEX_FOLDER)

    print(f"\n✅ Chatbot ready! Indexed {len(chatbot.documents)} chunks\n")

except Exception as e:
    print(f"❌ Error: {e}")
    raise

def ask(question: str):
    """Ask any question - AI will understand and answer"""
    result = chatbot.answer_question(question)

    print(f"\n{'='*80}")
    print(f"❓ {question}")
    print(f"{'='*80}")
    print(f"\n💡 {result['answer']}\n")

    if result['sources']:
        print(f"📚 Sources:")
        for i, src in enumerate(result['sources'], 1):
            print(f"  [{i}] {src['pdf']} (Page {src['page']})")

    print(f"{'='*80}\n")

    return result

print("🔍 Ask ANY question - the AI will understand!")
print("\nExamples:")
print("  ask('What is the revenue?')")
print("  ask('How much profit did they make?')")
print("  ask('Who runs the company?')")
print("  ask('What is their growth strategy?')")
print("  ask('What markets do they serve?')")

In [ ]:
ask('What is TechnologyOne total revenue for FY24?')
ask('What was the net profit?')
ask('Who is the CEO?')